# PRODUCER

# Análisis de Transacciones en la Plataforma Uber

## Universidad Tecnológica de Panamá  
### Facultad de Ingeniería de Sistemas Computacionales  
### Maestría en Analítica de Datos  

**Integrantes:**
- Ávila, Isaac  
- Sanjur, Andy

### 1. Descripción del caso simulado

En la última década, las plataformas de solicitud de viajes han adquirido una relevancia significativa en el ámbito de la movilidad urbana. Gracias a su practicidad, cada vez más usuarios y conductores optan por este tipo de soluciones de transporte privado. En una ciudad tan grande como Nueva York (NYC), pueden registrarse cientos de viajes por minuto, lo cual representa un reto importante para el análisis de datos en tiempo real mediante métodos convencionales, o bien vuelve ineficiente el análisis de datos históricos para el ajuste dinámico de tarifas según la demanda del momento.

El siguiente caso simulado presenta una serie de transacciones realizadas por la plataforma Uber. Cada registro corresponde a un viaje efectuado desde un punto A hasta un punto B, incluyendo su respectivo detalle. A partir de estos datos, se busca analizar los siguientes aspectos:

- ¿Cuáles son los puntos de partida más comunes?
- ¿Cuáles son los destinos más concurridos?
- ¿En qué días y a qué horas existe mayor frecuencia de solicitudes de viaje?
- Determinar cuáles son las matrículas con mayor cantidad de demoras.
- Determinar los minutos promedio de demora.

Para el análisis se utiliza un conjunto de datos en formato `.csv`, recopilado de la plataforma Kaggle:  
https://www.kaggle.com/datasets/ahmedramadan74/uber-nyc  

Este conjunto de datos incluye los siguientes campos relevantes para el caso de uso:

- **index_trip (int):** Número único del evento.
- **hvfhs_license_num (str):** Tipo de servicio contratado.
- **request_datetime (timestamp):** Fecha y hora en que se solicita el servicio.
- **pickup_datetime (timestamp):** Fecha y hora de inicio del viaje.
- **dropoff_datetime (timestamp):** Fecha y hora de finalización del viaje.
- **PULocationID (int):** Zona donde inicia el viaje.
- **DOLocationID (int):** Zona donde finaliza el viaje.
- **trip_miles (double):** Distancia total del viaje.
- **trip_time (int):** Duración del viaje en segundos.
- **base_passenger_fare (double):** Tarifa base del viaje.
- **tolls (double):** Total de peajes.
- **sales_tax (double):** Total de impuestos.
- **congestion_surcharge (double):** Costos adicionales por congestión vehicular.
- **airport_fee (double):** Tarifa adicional por aeropuerto (USD 2.50).
- **tips (double):** Propinas.
- **driver_pay (double):** Pago al conductor.
- **hour (double):** Hora de la solicitud.
- **year (double):** Año de la solicitud.
- **month (double):** Mes de la solicitud.
- **day (double):** Día de la solicitud.
- **on_time_pickup (int):** Indicador de llegada a tiempo.

---

### 2. Justificación del uso de Big Data en el problema

La alta demanda y el gran volumen de información que puede generar la aplicación durante las horas pico dificultan el análisis en tiempo real mediante herramientas tradicionales de analítica de datos. En este contexto, las oportunidades de negocio pueden perder relevancia si la información no es procesada y utilizada de manera inmediata.

Para este caso, con el objetivo de mantener la rapidez en el análisis y manejar eficientemente el volumen de datos que se generan cada minuto, se emplean tecnologías orientadas a Big Data, tales como **Apache Cassandra** para el almacenamiento distribuido de datos, **Apache Kafka** para la transmisión de datos en tiempo real (streaming) y **PySpark** para la transformación, consulta y exportación de la información analizada.


### 3. Proceso utilizado para el análisis

#### 3.1 Creación de la sesión de Spark

En esta etapa se importa la librería **SparkSession** y se crea la sesión de Apache Spark.  
Para ello, se configura el **host** y el **puerto** de conexión que permitirán establecer la comunicación con el entorno de procesamiento distribuido.

##### 3.1.1 Definición de constantes de configuración

Con el fin de mejorar la mantenibilidad y portabilidad del código, se definen constantes que agrupan los parámetros de configuración de la sesión de Spark y la conexión con la base de datos Apache Cassandra. Este enfoque permite modificar fácilmente los valores de conexión sin afectar la lógica del procesamiento del análisis.


In [1]:
# CONSTANTES
CASSANDRA_HOST = "100.68.89.127"
CASSANDRA_PORT = "9042"

APP_NAME = "Proyecto_Final_Uber_Trips"
CASSANDRA_CONNECTOR = "com.datastax.spark:spark-cassandra-connector_2.12:3.3.0"

CASSANDRA_CONNECTOR = "com.datastax.spark:spark-cassandra-connector_2.13:3.5.1"


In [2]:
from pyspark.sql import SparkSession

# Creación de SparkSession
spark = SparkSession.builder \
    .appName(APP_NAME) \
    .config("spark.jars.packages", CASSANDRA_CONNECTOR) \
    .config("spark.cassandra.connection.host", CASSANDRA_HOST) \
    .config("spark.cassandra.connection.port", CASSANDRA_PORT) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/admin/jupyter_venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/admin/.ivy2.5.2/cache
The jars for the packages stored in: /home/admin/.ivy2.5.2/jars
com.datastax.spark#spark-cassandra-connector_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6fa5f95d-b223-42f8-9900-445c22d9b95c;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.13;3.5.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.13;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.13;2.11.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	

#### 3.2 Lectura e importación del conjunto de datos

En esta etapa se realiza la lectura del conjunto de datos en formato `.csv` utilizando PySpark. Posteriormente, se visualizan las primeras cinco filas del conjunto de registros con el fin de validar la correcta carga de la información. Adicionalmente, se emplea la función `printSchema()` para inspeccionar la estructura del esquema y los tipos de datos asociados a cada columna.


In [3]:
raw_trips=spark.read.csv("data/sampling_trips.csv", header=True, inferSchema=True)
raw_trips.show(5)
print('Esquema utilizado en el conjunto de datos:')
raw_trips.printSchema()

25/12/15 20:56:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+----+------+-----+----+--------------+
|index_trip|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|hour|  year|month| day|on_time_pickup|
+----------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+-

#### 3.3 Integración con Cassandra

En esta etapa se realiza la creación y conexión al clúster de **Apache Cassandra**, el cual es utilizado como sistema de almacenamiento distribuido. Se define el **keyspace** denominado `trips` y se crea la tabla `uber_trips`, estructurada de acuerdo con las columnas del conjunto de datos analizado. Finalmente, se efectúa la escritura de la información en Cassandra, persistiendo los datos previamente cargados y procesados mediante Apache Spark.


In [8]:
from cassandra.cluster import Cluster
from cassandra.policies import RoundRobinPolicy

KEYSPACE = "trips"
TABLE = "uber_trips"
REPLICATION_FACTOR = 1
BATCH_SIZE = 10000

# =========================
# Conexión a Cassandra
# =========================
cluster = Cluster(
    contact_points=["127.0.0.1"],
    port=CASSANDRA_PORT,
    load_balancing_policy=RoundRobinPolicy()
)

session = cluster.connect()

BORRAR_DATOS = True

if BORRAR_DATOS:
    session.execute("TRUNCATE trips.uber_trips;")
    print("Datos de la tabla 'uber_trips' eliminados correctamente.")
else:
    print("Borrado deshabilitado. No se realizaron cambios.")

# =========================
# Crear Keyspace (si no existe)
# =========================
session.execute(f"""
CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
WITH replication = {{
    'class': 'SimpleStrategy',
    'replication_factor': {REPLICATION_FACTOR}
}};
""")

session.set_keyspace(KEYSPACE)

# =========================
# Crear tabla (si no existe)
# =========================
session.execute(f"""
CREATE TABLE IF NOT EXISTS {TABLE} (
    index_trip int PRIMARY KEY,
    hvfhs_license_num text,
    dispatching_base_num text,
    originating_base_num text,
    request_datetime timestamp,
    on_scene_datetime timestamp,
    pickup_datetime timestamp,
    dropoff_datetime timestamp,
    pulocationid int,
    dolocationid int,
    trip_miles float,
    trip_time int,
    base_passenger_fare float,
    tolls float,
    bcf float,
    sales_tax float,
    congestion_surcharge float,
    airport_fee float,
    tips float,
    driver_pay float,
    shared_request_flag text,
    shared_match_flag text,
    access_a_ride_flag text,
    wav_request_flag text,
    wav_match_flag text,
    hour float,
    year float,
    month float,
    day float,
    on_time_pickup int
);
""")

print("Keyspace y tabla verificados.")

# =========================
# Verificar si la tabla ya tiene datos
# =========================
row_check = session.execute(
    f"SELECT index_trip FROM {TABLE} LIMIT 1"
).one()
if row_check:
    print("La tabla ya contiene datos. No se realiza inserción.")
else:
    print("Tabla vacía. Iniciando carga de datos desde Spark...")

    (
        raw_trips
        .repartition(max(1, raw_trips.count() // BATCH_SIZE))
        .write
        .format("org.apache.spark.sql.cassandra")
        .mode("append")
        .options(table=TABLE, keyspace=KEYSPACE)
        .save()
    )

    print("Datos cargados correctamente en Cassandra.")


Datos de la tabla 'uber_trips' eliminados correctamente.
Keyspace y tabla verificados.
La tabla ya contiene datos. No se realiza inserción.


[Stage 8:==================================>                   (279 + 14) / 435]

#### 3.4 Consulta de la tabla en Cassandra

En esta etapa se valida la conexión con **Apache Cassandra** mediante la lectura de los datos almacenados desde Apache Spark. Se realiza la consulta de la tabla `uber_trips`, perteneciente al keyspace `trips`, y los resultados se cargan en un DataFrame denominado `df_raw`. Finalmente, se visualizan las primeras diez filas con el objetivo de verificar la correcta persistencia y recuperación de la información.


In [ ]:
spark.conf.get("spark.cassandra.connection.host")

In [ ]:
from pyspark.sql import SparkSession
df_raw=spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="uber_trips", keyspace="trips") \
    .load()\
 
df_raw.show(10)

<b>3.5. Transformación del conjunto de datos</b>
<br>
Se realiza la lectura del archivo `.csv` con la información de las ubicaciones y se aplican transformaciones de tipo `JOIN` entre tablas. Posteriormente, se ejecutan cálculos entre columnas para determinar la comisión de la plataforma y se generan agregaciones temporales. Finalmente, los datos procesados y agregados son exportados en formato `.csv`.

In [ ]:
from pyspark.sql.functions import col

# Lectura del archivo de ubicaciones
loc = (
    spark.read
    .csv("taxi_zone_lookup.csv", header=True, inferSchema=True)
    .drop("Borough", "service_zone")
)

loc_pickup = loc.withColumnRenamed("Zone", "pickup_zone")
loc_dropoff = loc.withColumnRenamed("Zone", "delivery_zone")

# Limpieza de registros nulos
df_clean = df_raw.dropna()

# Cálculo de ganancias de la plataforma
df_clean = df_clean.withColumn(
    "uber_sales",
    col("base_passenger_fare") + col("tips") - col("driver_pay")
)

# Join con zona de origen
df_clean = (
    df_clean
    .join(loc_pickup, df_clean.PULocationID == loc_pickup.LocationID, "left")
    .drop("LocationID")
)

# Join con zona de destino
df_clean = (
    df_clean
    .join(loc_dropoff, df_clean.DOLocationID == loc_dropoff.LocationID, "left")
    .drop("LocationID")
)

# Agregación de ganancias por tiempo
df_agg_uber_sales = (
    df_clean
    .groupBy("year", "month", "day", "hour")
    .sum("uber_sales")
)

# Escritura de resultados
df_clean.write.mode("append").option("header", True).csv("Proyecto_Final/clean_data.csv")
df_agg_uber_sales.write.mode("append").option("header", True).csv("Proyecto_Final/aggregated_data.csv")

<b>3.6 Creación de la tabla en Cassandra de datos procesados</b>
<br>
Utilizando el mismo keyspace en Cassandra, se crea una nueva tabla denominada `uber_trips_clean`, la cual incluye las columnas generadas tras el proceso de transformación. Posteriormente, la tabla es poblada con el dataframe resultante.


In [ ]:
from cassandra.cluster import Cluster
from cassandra.policies import RoundRobinPolicy

# Configuración Cassandra (capa prod)
PROD_CASSANDRA_HOST = "127.0.0.1"
PROD_CASSANDRA_PORT = 9042
PROD_KEYSPACE = "trips"

# Cluster y sesión para datos procesados (prod)
prod_cluster = Cluster(
    contact_points=[PROD_CASSANDRA_HOST],
    port=PROD_CASSANDRA_PORT,
    load_balancing_policy=RoundRobinPolicy()
)

prod_session = prod_cluster.connect()
prod_session.set_keyspace(PROD_KEYSPACE)

# Creación de la tabla de datos procesados
prod_session.execute("""
CREATE TABLE IF NOT EXISTS uber_trips_prod (
    index_trip int PRIMARY KEY,
    hvfhs_license_num text,
    dispatching_base_num text,
    originating_base_num text,
    request_datetime timestamp,
    on_scene_datetime timestamp,
    pickup_datetime timestamp,
    dropoff_datetime timestamp,
    PULocationID int,
    DOLocationID int,
    trip_miles float,
    trip_time int,
    base_passenger_fare float,
    tolls float,
    bcf float,
    sales_tax float,
    congestion_surcharge float,
    airport_fee float,
    tips float,
    driver_pay float,
    shared_request_flag text,
    shared_match_flag text,
    access_a_ride_flag text,
    wav_request_flag text,
    wav_match_flag text,
    hour float,
    year float,
    month float,
    day float,
    on_time_pickup int,
    uber_sales float,
    pickup_zone text,
    delivery_zone text
);
""")

print("Tabla 'uber_trips_prod' creada usando la sesión prod_session.")


In [ ]:
df_merged4.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table="uber_trips_prod", keyspace="trips") \
    .save()

print("Datos cargados en Cassandra (tabla uber_trips_prod).")


<b>3.7.a Configuración del productor en Kafka (Tabla cruda)</b>
<br>
Se establece la conexión con Cassandra para consultar la tabla `uber_trips`, almacenando los registros en una variable. Posteriormente, se importan las librerías necesarias para Kafka y JSON con el fin de serializar los datos y establecer la conexión con el servicio. Finalmente, se empaquetan las columnas requeridas en un diccionario y se envían al tópico `uber_trips` con un intervalo de un segundo entre cada transacción.

In [ ]:
from cassandra.cluster import Cluster

RAW_CASSANDRA_HOST = "127.0.0.1"
RAW_KEYSPACE = "trips"
RAW_TABLE = "uber_trips"

raw_cluster = Cluster([RAW_CASSANDRA_HOST])
raw_session = raw_cluster.connect(RAW_KEYSPACE)

raw_rows = raw_session.execute(f"SELECT * FROM {RAW_TABLE}")

from kafka import KafkaProducer
import json
from datetime import datetime, timezone
import time

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)

In [ ]:
for row in rows:
    row_dict = {
        "index_trip":row.index_trip,
        "request_datetime":row.request_datetime.replace(tzinfo=timezone.utc).isoformat(),
        "pickup_datetime":row.pickup_datetime.replace(tzinfo=timezone.utc).isoformat(),
        "dropoff_datetime":row.dropoff_datetime.replace(tzinfo=timezone.utc).isoformat(),
        "PULocationID":row.PULocationID,
        "DOLocationID":row.DOLocationID,
        "base_passenger_fare":row.base_passenger_fare,
        "tips":row.tips,
        "driver_pay":row.driver_pay,
        "hour":row.hour,
        "year":row.year,
        "month":row.month,
        "day":row.day,
        "on_time_pickup":row.on_time_pickup
    }
    
    producer.send('uber_trips',row_dict)
    print('enviado:', row_dict)
    time.sleep(5)   

<b>3.7.b Configuración del productor en Kafka (Tabla procesada)</b>
<br>
Se establece la conexión con Cassandra para consultar la tabla `uber_trips_prod`, almacenando los registros en una variable. Posteriormente, se importan las librerías necesarias para Kafka y JSON con el fin de serializar los datos y establecer la conexión con el servicio. Finalmente, se empaquetan las columnas requeridas en un diccionario y se envían al tópico `uber_trips_prod` con un intervalo de un segundo entre cada transacción.


In [ ]:
from cassandra.cluster import Cluster
from kafka import KafkaProducer
import json
from datetime import timezone
import time

# Configuración Cassandra (capa PROD)
PROD_CASSANDRA_HOST = "127.0.0.1"
PROD_KEYSPACE = "trips"
PROD_TABLE = "uber_trips_prod"

prod_cluster = Cluster([PROD_CASSANDRA_HOST])
prod_session = prod_cluster.connect(PROD_KEYSPACE)

prod_rows = prod_session.execute(f"SELECT * FROM {PROD_TABLE}")

# Configuración del productor Kafka
producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda m: json.dumps(m).encode("utf-8")
)

In [ ]:
for row in prod_rows:
    row_dict = {
        "index_trip": row.index_trip,
        "request_datetime": row.request_datetime.replace(tzinfo=timezone.utc).isoformat(),
        "pickup_datetime": row.pickup_datetime.replace(tzinfo=timezone.utc).isoformat(),
        "dropoff_datetime": row.dropoff_datetime.replace(tzinfo=timezone.utc).isoformat(),
        "PULocationID": row.pulocationid,
        "DOLocationID": row.dolocationid,
        "base_passenger_fare": row.base_passenger_fare,
        "tips": row.tips,
        "driver_pay": row.driver_pay,
        "hour": row.hour,
        "year": row.year,
        "month": row.month,
        "day": row.day,
        "on_time_pickup": row.on_time_pickup,
        "uber_sales": row.uber_sales,
        "pickup_zone": row.pickup_zone,
        "delivery_zone": row.delivery_zone
    }

    producer.send("uber_trips_prod", row_dict)
    print("enviado:", row_dict)
    time.sleep(5)
